# Principal Component Analysis with Ship Data
**Before you begin.**
You will edit this Colab notebook by adding/changing code and adding text to answer questions and provide explanations and insights.  When you are finished, you can print to PDF in order to easily generate a report.  To earn full points,  
*   try your best to avoid code that extends past the vertical bar in code cells.  It will not print well.
*   print text along with the answer/explanation, if answering/explaining with code.  For example, use `print(f'The bias is {bias}.')` instead of `print(bias)`.  This way, your instructor knows that you know how to interpret what you are seeing,
*   use italicized text if answering/explaining with text.  This will help your answers stand out, and
*   identify and explain some key observations that were not explcitily asked for in this assignment.  Sometimes the instructor wants to see if you know what to look for on your own.

You can add cells of either text or code by hovering over the horizontal edge of the cell just before where you want to add a new cell.  Click either "+ text" or "+ code", as appopriate.  You may have to click in the previous cell in order to see "+ text" or "+ code"


---



**Assignment summary.** This assignment will have you investigate principal component analysis (PCA) by reducing the dimension of the [San Francisco ship aerial imagery dataset](https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery).  As opposed to using the black-box function, you will code the PCA algorithm from scratch, calculating the covariance matrix and employing eigenvalue decomposition and projection methods. You will observe the retained variance of each principal component and the reconstruction of images from their reduced-dimension representation, using reconstruction error to build an anomaly detector.  

The black and white ship dataset is the same in this assignment as in the previous assignments.  You will compare your dimensionally reduced dataset to the one provided to you in those previous assignments.

---


**Import the data.** Connect this Colab notebook to your Google Drive.  This code is complete.

In [ ]:
# Given your permission, this will connect your notebook to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Load in the ship data from Google Drive and print out the number of images.  The name of the folder containing all the ships should be `shipdata_MLcourse`, otherwise you need to change the code below accordingly.  The variable `img_data` contains all of the pixel information for each of the images.  This code is complete.

In [ ]:
# Now the ship data can be loaded into the notebook (presuming you uploaded..
# ...the data previously into your Google Drive)
import os, pickle

my_path = '/content/drive/My Drive/shipdata_MLcourse'
with open(os.path.join(my_path, 'img_data.pickle'), 'rb') as handle:
  img_data = pickle.load(handle)

print(f'The number of images is {len(img_data)}.') # print the number of images
print(f'Each image is a vector of length {len(img_data[0])}.')
print(f'The shape of img_data is {img_data.shape}.')          

Get the targets vector `y`: 1 if the image contains a ship, 0 for no ship (or partial ship). This code is complete.


In [ ]:
with open(os.path.join(my_path, 'targets.pickle'), 'rb') as handle:
  y = pickle.load(handle)
  print(f'The length of y is {len(y)}.')

Check out some of the ship images!  Each image is a vector of length 6400 that can be reshaped into 80 by 80 to make a small picture.  Change the value of `my_index` to view different images.  Remember that Python indexing starts at zero.  The code will print out the target as well.  Images towards the end of the dataset (larger indices) contain the ships. The code is complete.

In [ ]:
from google.colab.patches import cv2_imshow

my_index = 1000        # change this number
img = img_data[my_index].reshape((80,80))
cv2_imshow(img)  # display image

if y[my_index] == 0:
  print('This image is no ship or partial ship.')
else:
  print('This image is ship.')

**Use PCA to reduce the dimension of the ship data.** The data must be centered in order to apply the PCA algorithm.  Use the `mean` function from the `numpy` library to get the mean of each of the 6400 features (pixels).  Then, subtract the means from `img_data` with Python broadcasting to center the data.

Finally, use the [cov](https://numpy.org/doc/stable/reference/generated/numpy.cov.html) function from the `numpy` library to calculate the covariance matrix of the centered data.  You will need to change one of the function's default parameters to account for the fact that variables (features) are columns, not rows.  When you are finished, the covariance matrix should be stored as variable `Sigma`.

In [ ]:
import numpy as np
# TODO: Use mean to calculate the means of all 6400 features
means = 
# TODO: Subtract means from img_data
img_data_centered = 
# TODO: Use cov to find the covariance matrix
Sigma = 

Next, we use the [eig](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html) function from the `linalg` sub-library of the `numpy` library to find the eigenvalues and eigenvectors of the covariance matrix.  Then we sort the eigenvalues in descending order and their corresponding vectors.  The code below is complete, and it may take a few minutes to execute.  Disregard any warnings you may receive related to the imaginary parts of the eignevalues, which are practically zero.

In [ ]:
vals, vecs = np.linalg.eig(Sigma)
idx = vals.argsort()[::-1]   
sorted_vals = vals[idx]
sorted_vecs = vecs[:,idx]

Now the eigenvalues $\lambda_j$ are sorted so that $\lambda_1 > \lambda_2 > ... > \lambda_{6400}$.  Recall the $j$th eigenvalue is the data variance in the direction of the $j$th principal component.  So the quantity $
\frac{\lambda_1}{\sum\lambda}$ is the fraction of the total variance explained by the 1st principal component.  The quantity $
\frac{\lambda_1 + \lambda_2}{\sum\lambda_i}$ is the fraction of the total variance explained by the first two principal components (the cumulative fractional variance), and so on.  Thus, the cumulative fractional variance explained by the first $j$ principal components is
$$
\frac{\sum_{p=1}^{j}\lambda_{p}}{\sum\lambda}.
$$  

Make a plot showing the cumulative fractional variance versus $j$, for $j = 1, ..., 6400$ (a "scree plot").  Use the `sum` and the `cumsum` functions from the `numpy` library to help you.  Use the `plot` function from `matplotlib` to generate the plot.

Disregard any warnings you may receive related to the imaginary parts of the eignevalues, which are practically zero.

Describe what you seen in your plot.  How much fractional variance is accounted for by the first 300 PCs?

In [ ]:
import matplotlib.pyplot as plt
# TODO: Generate the scree plot.  Use sum, cumsum, and plot.


# TODO: How much fractional variance is accounted for by the first 300 PCs?
num_PCs = 300



Reduce the dimension of `img_data_centered` from 6400 to 300.  Construct a matrix `W` 6400 by 300 in size whose columns are the first 300 eigenvectors of the covariance matrix.  Then, multiply `img_data_centered` by `W`.  This will be similar to the reduced dataset `X` you were supplied in the last two assignments!  The code below will load in that dataset, and then you should compare some elements of the two.  What do you notice? By the way, the data matrix `X` was generated using the black-box function [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [ ]:
# TODO: Generate W and reduce img_data_centered
W = 


# ---This loads in the dataset X------
with open(os.path.join(my_path, 'X_pca.pickle'), 'rb') as handle:
  X = pickle.load(handle)

# TODO: Compare some elements from the transformed matrix to X


**Examine PCA reconstruction error.**  Information is lost in the process of compressing each image from 6400 values to 300.  Hopefully, the losses are not too great--Here you will reconstruct the images, calculate their reconstruction errors, and view some of the images versus their reconstructions.

In [ ]:
# TODO: Reconstruct the images by multiplying the reduced dataset by the...
# ...transpose of W and then adding the means back.
img_data_reconstructed = 

my_index = 39        # change this number to change the image displayed

# ---View the reconstructed image and its original-------
img = img_data_reconstructed[my_index].reshape((80,80))
cv2_imshow(img)  # display image
img = img_data[my_index].reshape((80,80))
cv2_imshow(img)  # display image

Suppose the $i$th original image with 6400 pixels is represented as vector
$$
\mathbf{x(i)} = [x_1(i),~x_2(i),~...,~x_{6400}(i)],
$$
and the corresponding reconstructed image as 
$$
\mathbf{\hat{x}(i)} = [\hat{x}_1(i),~\hat{x}_2(i),~...,~\hat{x}_{6400}(i)].
$$
The pixel-wise mean-squared error between the two is a good measure of how different the images are, and how poorly the $i$th image was reconstructed:
$$
error(i) = \frac{1}{6400}\sum_{j=1}^{6400}(x_j(i) - \hat{x}_j(i))^2.
$$
Use the `mean` function from `numpy` to calculate the error for all 4000 images.  The variable `errors` should contain the reconstruction error of all 4000 images.  When finished, the code will plot a histogram of the error distribution.  Again, you can ignore any warnings regarding complex values.

In [ ]:
# TODO: Determine the reconstruction error of each image 
errors = 
plt.hist(errors)
plt.xlabel('PCA reconstruction error')
plt.ylabel('number of images')

What would be the reconstruction error of an image that is not from this dataset?  One that is not an aerial image at all?  You will find out below, but first take a moment to observe the histogram from the previous cell.  What was the largest reconstruction error in the dataset?

The code below creates and displays an image that is completely different from any in the dataset.  We will call it an outlier.  The code is complete.

In [ ]:
outlier = np.zeros((80,80))
outlier[20:30,20:30] = 255
outlier[20:30, 50:60] = 255
outlier[50:60, 20:60] = 255
cv2_imshow(outlier)  

We will reshape the image to a row vector of length 6400.  Then, you need to complete the code to use PCA to reduce the (mean-centered) vector to 300 dimensions, reconstruct the image, and calculate the reconstruction error.  What is the error?  You should see that it is much larger than any reconstruction error in the dataset.  So, you have just programmed an anomaly (outlier) detector!  We can use PCA to automatically detect anomalous data since an anomalous datapoint is one whose PCA reconstruction error is exceptionally large.  Neat!  Such an algorithm is important, for example, in banking to detect credit card fraud, or in electric machinery to detect equipment failure.

In [ ]:
outlier_as_vec = outlier.reshape((1,6400))     # reshape the image to a vector
# TODO: outlier_reduced is a PCA-reduced vector of length 300.  Don't forget ...
# ... to subtract means
outlier_reduced = 
# TODO: reconstruct the image to a length 6400 vector.  Multiply by the ...
# ...transpose of W and add back the means
outlier_reconstructed = 
# TODO: calculate and observe the mean reconstruction error
error_outlier = 


This is the end of what is required.  You can easily generate a report by navigating to File -> Print and then selecting to save to PDF.  You will turn in this PDF.  Be sure that the report shows your code's output (that is, you've run each cell), and your notes/explanations are italicized.  Continue on for extra insights.

**Go further.** Since each eigenvector of the covariance matrix is itself a vector of length 6400, each may be reshaped into images and displayed.  After reshaping, the values must be scaled in the range 0 - 255 integers to be displayed as an 8-bit image.  

The code below is complete and displays the first 1000 "eigenships".  Run it.  What do you notice?  Is there any structure to the images?  Explain what you see.

In [ ]:
for index in range(1000):
  img = np.real(sorted_vecs[:,index].reshape((80,80)))
  mx = np.min(np.real(img))
  mn = np.max(np.real(img))
  new_arr = ((img + mn) * (1/mx) * 255).astype('uint8')
  cv2_imshow(new_arr)